<img src="http://milepro.com/wp-content/uploads/2014/01/Travel-Credit-Cards-1024x606.jpg" style="width:200px; float: left; padding-right: 10px"/>
<h2 style="font-face: verdana; font-size: 32px;">Predict credit card customer churn<br>with IBM Watson Machine Learning</h2>
<h3 style="font-face: verdana; font-size: 16px;">Part 2: Create Churn Model</h3>




### 1. Load the data into a dataframe ##
-------------------------------------
<p>In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration.</p>
<p>Load the data from the DB2 table to the Spark DataFrame using JDBC driver.</p>

In [1]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.util import MLUtils

In [2]:
from io import StringIO
import requests
import json
import pandas as pd
import numpy as np

In [3]:
import jaydebeapi, sys

#Enter the values for you database connection
dsn_database = "XXXX"            # e.g. “BLUDB” Name of the database
dsn_hostname = "X.XX.XXX.XX" # e.g.: “bluemix05.bluforcloud.com”
dsn_port = "XXXXX"                # e.g. “50000" Database port number
dsn_uid = "XXXXXXXX"        # e.g. “dash104434" User id
dsn_pwd = "XXXXXXXX"       # e.g. “7dBZ3jWt9xN6$o0JiX!m” User password for the database

connection_string='jdbc:db2://'+dsn_hostname+':'+dsn_port+'/'+dsn_database
if (sys.version_info >= (3,0)):
    conn = jaydebeapi.connect("com.ibm.db2.jcc.DB2Driver", connection_string, [dsn_uid, dsn_pwd])
else:
    conn = jaydebeapi.connect("com.ibm.db2.jcc.DB2Driver", [connection_string, dsn_uid, dsn_pwd])

curs = conn.cursor()
curs.execute("select * from XXX.XXXXXXXX")
list1 = curs.fetchall()
print(list1[0])

(1009520380, u'F', 59, 1, 0.0, 18267.0, 0, 0, 7957.1406, 0.654795, 239, 33.0, 2, u'ME', u'High school graduate', 0, u'FALSE')


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(list1, ['CUST_ID', 'SEX', 'AGE', 'EDUCATION', 'INVESTMENT', 'INCOME', 'ACTIVITY', 'CHURN', 'YRLY_AMT', 'AVG_DAILY_TX', 'YRLY_TX', 'AVG_TX_AMT', 'NEGTWEETS', 'STATE','EDUCATION_GROUP', 'TWITTERID', 'CHURN_LABEL'])
df.take(5)

[Row(CUST_ID=1009520380, SEX=u'F', AGE=59, EDUCATION=1, INVESTMENT=0.0, INCOME=18267.0, ACTIVITY=0, CHURN=0, YRLY_AMT=7957.1406, AVG_DAILY_TX=0.654795, YRLY_TX=239, AVG_TX_AMT=33.0, NEGTWEETS=2, STATE=u'ME', EDUCATION_GROUP=u'High school graduate', TWITTERID=0, CHURN_LABEL=u'FALSE'),
 Row(CUST_ID=1009520400, SEX=u'M', AGE=50, EDUCATION=1, INVESTMENT=0.0, INCOME=17867.0, ACTIVITY=2, CHURN=0, YRLY_AMT=7946.078, AVG_DAILY_TX=0.660274, YRLY_TX=241, AVG_TX_AMT=32.0, NEGTWEETS=4, STATE=u'UT', EDUCATION_GROUP=u'High school graduate', TWITTERID=0, CHURN_LABEL=u'FALSE'),
 Row(CUST_ID=1009520410, SEX=u'F', AGE=53, EDUCATION=1, INVESTMENT=0.0, INCOME=17576.0, ACTIVITY=1, CHURN=0, YRLY_AMT=7961.8086, AVG_DAILY_TX=0.663014, YRLY_TX=242, AVG_TX_AMT=32.0, NEGTWEETS=4, STATE=u'AK', EDUCATION_GROUP=u'High school graduate', TWITTERID=0, CHURN_LABEL=u'FALSE'),
 Row(CUST_ID=1009520420, SEX=u'F', AGE=48, EDUCATION=3, INVESTMENT=90419.0, INCOME=111569.0, ACTIVITY=5, CHURN=0, YRLY_AMT=26803.0, AVG_DAILY_TX=0



### 1.2 Select Churn Data for the Model ###
<p>Select AGE, ACTIVITY, EDUCATION, SEX, STATE, NEGTWEETS, INCOME, CHURN from the churnDataRaw dataframe.</p>

In [5]:
churnData = df.select('AGE', 'ACTIVITY', 'EDUCATION', 'SEX', 'STATE', 'NEGTWEETS', 'INCOME', 'CHURN')
churnData.head()

Row(AGE=59, ACTIVITY=0, EDUCATION=1, SEX=u'F', STATE=u'ME', NEGTWEETS=2, INCOME=18267.0, CHURN=0)




## 2. Create an Apache Spark machine learning model ##
-------------------------------------
<p>Prepare data, create an Apache Spark machine learning pipeline, and train a model.</p>




### 2.1 Prepare the Data ###
<p>In this subsection you will split your data into: train, test and predict datasets.</p>

In [6]:
trainDF, validateDF, testDF = churnData.randomSplit([.7,.15,.15])
trainDF.head()

Row(AGE=20, ACTIVITY=2, EDUCATION=2, SEX=u'M', STATE=u'CA', NEGTWEETS=6, INCOME=25513.0, CHURN=1)

In [7]:
testDF.head()

Row(AGE=20, ACTIVITY=3, EDUCATION=1, SEX=u'F', STATE=u'ID', NEGTWEETS=3, INCOME=20549.0, CHURN=0)

In [8]:
validateDF.head()

Row(AGE=22, ACTIVITY=1, EDUCATION=4, SEX=u'M', STATE=u'PA', NEGTWEETS=3, INCOME=58691.0, CHURN=0)




### 2.2 Create pipeline and train a model ###
<p>In this section you will create an Apache® Spark machine learning pipeline and then train the model.</p>

In [9]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In [10]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

genderIndexer = StringIndexer(inputCol="SEX", outputCol="gender_code")
stateIndexer = StringIndexer(inputCol="STATE", outputCol="state_code")
featuresAssembler = VectorAssembler(
    inputCols=["AGE", "ACTIVITY","EDUCATION","NEGTWEETS","INCOME","gender_code","state_code"],
    outputCol="features")

<p>Next, define estimators you want to use for classification. Logistics Regression is used in the following example.</p>

In [11]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(regParam=0.01, labelCol="CHURN", featuresCol="features")

<p>Setup a Cognitive Assistant for Data Scientists - predict model performance based on sampled data</p>

In [12]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [13]:
pipeline = Pipeline(stages=[genderIndexer, stateIndexer, featuresAssembler, lr])
model = pipeline.fit(trainDF)

<p>You can check your model accuracy now. To evaluate the model, use test data.</p>

In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

predictions = model.transform(testDF)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="CHURN", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)
print "Accuracy = " + str(accuracy)
print "Test Error = " + str(1.0 - accuracy)

Accuracy = 0.986998916576
Test Error = 0.0130010834236


In [15]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.regression import LabeledPoint

In [16]:
predictionsAndLabels = predictions.rdd.map(lambda r: (float(r.prediction),float(r['CHURN'])))
predictionsAndLabels.take(2)

[(0.0, 0.0), (1.0, 1.0)]

In [17]:
# Instantiate metrics object
metrics = BinaryClassificationMetrics(predictionsAndLabels)

In [18]:
# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

Area under PR = 0.978145720041
Area under ROC = 0.977936415231



<p>Create a true positive and false positive rate</p>

In [19]:
import numpy as np
pR = [(1.0,1.0)]
for i in np.arange(0.0, 1.01, 0.01):
    preds = predictions.rdd.map(lambda r: (float(r['probability'][1]>i),float(r['CHURN']),r['probability']))
    tp = float(preds.filter(lambda r: r[0] == 1.0 and r[1] == 1.0).count())
    fp = float(preds.filter(lambda r: r[0] == 1.0 and r[1] == 0.0).count())
    fn = float(preds.filter(lambda r: r[0] == 0.0 and r[1] == 1.0).count())
    tn = float(preds.filter(lambda r: r[0] == 0.0 and r[1] == 0.0).count())
    if tp == 0.0 and fp == 0.0:
        pR.append((0.0,0.0))
    else:
        recall = tp/(tp+fn)
        falsePosRate = fp/(fp+tn)
        pR.append((falsePosRate,recall))
pR.append((0.0,0.0))


<p>Load the rocCurve into a dataframe</p>

In [20]:
import seaborn as sns
import pandas as pd
%matplotlib inline
rocDF = pd.DataFrame(pR)
rocDF = rocDF.rename(index=str, columns={0: "FalsePositiveRate", 1: "TruePositiveRate"})




## 3. Watson Machine Learning - Use Repository service to save model. ##
-------------------------------------

In [21]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
from repository.mlrepository import MetaProps, MetaNames

ml_repository_client = MLRepositoryClient()

In [22]:
model_artifact = MLRepositoryArtifact(model, training_data=trainDF, name="Banking customer Churn")

saved_model = ml_repository_client.models.save(model_artifact)

print saved_model.meta.available_props()
print
print "modelType: " + saved_model.meta.prop("modelType")
print "trainingDataSchema: " + str(saved_model.meta.prop("trainingDataSchema"))
print "creationTime: " + str(saved_model.meta.prop("creationTime"))
print "modelVersionHref: " + saved_model.meta.prop("modelVersionHref")
print "label: " + saved_model.meta.prop("label")

Using TensorFlow backend.
/opt/conda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['inputDataSchema', 'description', 'modelVersionHref', 'authorUID', 'creationTime', 'lastUpdated', 'label', 'trainingDataSchema', 'version', 'modelType', 'runtime']

modelType: spark-2.0
trainingDataSchema: [{'type': 'bigint', 'name': 'AGE'}, {'type': 'bigint', 'name': 'ACTIVITY'}, {'type': 'bigint', 'name': 'EDUCATION'}, {'type': 'string', 'name': 'SEX'}, {'type': 'string', 'name': 'STATE'}, {'type': 'bigint', 'name': 'NEGTWEETS'}, {'type': 'double', 'name': 'INCOME'}, {'type': 'bigint', 'name': 'CHURN'}]
creationTime: 1523925534681
modelVersionHref: https://dsxl-api.ibm-private-cloud.svc.cluster.local/v3/project/Bank_Cust_Churn_Analysis/models/Banking%20customer%20Churn/2
label: CHURN


<div class="alert alert-block alert-info"> Tip: modelVersionHref is our model unique indentifier in the Watson Machine Learning repository.</div> 

<p>Get saved model metadata from Watson Machine Learning.</p>

In [23]:
saved_model.meta.available_props()

['inputDataSchema',
 'description',
 'modelVersionHref',
 'authorUID',
 'creationTime',
 'lastUpdated',
 'label',
 'trainingDataSchema',
 'version',
 'modelType',
 'runtime']




### 3.2 Load model and make predictions ##
-------------------------------------

In [24]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)

In [25]:
print(loadedModelArtifact.name)
print(saved_model.uid)

Banking customer Churn
/user-home/999/DSX_Projects/Bank_Cust_Churn_Analysis/models/Banking customer Churn/2


In [26]:
predictions = loadedModelArtifact.model_instance().transform(validateDF)


In [27]:
predictions.select("AGE","ACTIVITY","EDUCATION","SEX","STATE","NEGTWEETS","INCOME","CHURN").show()


+---+--------+---------+---+-----+---------+-------+-----+
|AGE|ACTIVITY|EDUCATION|SEX|STATE|NEGTWEETS| INCOME|CHURN|
+---+--------+---------+---+-----+---------+-------+-----+
| 22|       1|        4|  M|   PA|        3|58691.0|    0|
| 23|       3|        4|  M|   PA|        3|21773.0|    0|
| 24|       5|        4|  M|   VA|        5|17830.0|    0|
| 25|       2|        4|  M|   FL|        4|59970.0|    0|
| 25|       2|        4|  M|   ID|       13|17191.0|    1|
| 25|       5|        1|  F|   MO|        4|16043.0|    0|
| 26|       0|        4|  M|   MI|       16|13552.0|    1|
| 26|       4|        1|  F|   ID|        3|18071.0|    0|
| 27|       2|        1|  M|   NM|        3|21619.0|    0|
| 27|       2|        4|  M|   WA|       11|29404.0|    1|
| 28|       2|        1|  F|   MI|        8|25604.0|    1|
| 28|       2|        1|  M|   ID|        7|19226.0|    1|
| 28|       5|        4|  F|   WA|        9|20550.0|    1|
| 29|       2|        1|  M|   CO|        3|22616.0|    